# Linear diffusion

In this example we show how to solve the following problem using PyBaMM:

$$c_t = D c_{xx}, \quad \left. c_x \right|_{x=0} = 0, \quad \left. -Dc_x \right|_{x=1}=j, \quad \left. c \right|_{t=0}=c_0,$$

where $D$ is the (constant) diffusivity, $c_0$ the initial concentration, and $j(t)$ the prescribed flux (which may be a function of time). We solve the problem on the interval $[0, 1]$. We assume that the problem has been non-dimensionalised and therefore do not worry about units.

## Building the model
As always, we start by importing PyBaMM, along with any other packages we require.

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

We then load up an instance of the `pybamm.BaseModel` class.

In [2]:
model = pybamm.BaseModel()

We now define the model variables and parameters. We also define the spatial variable $x$ here. Since we are solving in 1D we have decided to call the domain "rod", but we could name it anything we like. Note that in PyBaMM variables and parameters can be given useful and meaningful names, such as "Concentration", so that they can be easily referred to later. Values for the parameters will be provided later.

In [3]:
x = pybamm.SpatialVariable("x", domain="rod", coord_sys="cartesian")
c = pybamm.Variable("Concentration", domain="rod")
D = pybamm.Parameter("Diffusivity")
c0 = pybamm.Parameter("Initial concentration")

The flux $j$ may depend on time, so we make it a `FunctionParameter`, The time variable is already defined as `pybamm.t`

In [4]:
j = pybamm.FunctionParameter("Applied flux", {"Time": pybamm.t})

Now that we have defined the variables, we can write down the model equations and add them to the `model.rhs` dictionary. This dictionary stores the right hand sides of any time-dependent differential equations (ordinary or partial). The key is the variable inside the time derivative (in this case $c$).

In [5]:
N = - D * pybamm.grad(c)  # The flux 
dcdt = - pybamm.div(N)  # The right hand side of the PDE
model.rhs = {c: dcdt}  # Add to model

We now add the boundary conditions into the `model.boundary_conditions` dictionary. The keys of the dictionary indicate which end of the boundary the condition is applied to (in 1D this can be "left" or "right"), the entry is then give as a tuple of the value and type.

In [6]:
model.boundary_conditions = {
    c: {
        "left": (pybamm.Scalar(0), "Neumann"),
        "right": (-j / D, "Neumann"),
    }
}

We also need to add the initial conditions to the `model.initial_conditions` dictionary.

In [7]:
model.initial_conditions = {c: c0}

Finally, we add any output variables to the `model.variables` dictionary. These variables can be easily accessed after the model has been solved. You can add any variables of interest to this dictionary. Here we have added the concentration, surface concentration, flux, and time.

In [8]:
model.variables = {
    "Concentration": c, 
    "Surface concentration": pybamm.surf(c),
    "Applied flux": j,
    "Time": pybamm.t,
}

## Using the model
Now that the model has been constructed we can go ahead and define our geometry and parameter values. We start by defining the geometry for our "rod" domain. We need to define the spatial direction(s) in which spatial operators act (such as gradients). In this case it is simply $x$. We then set the minimum and maximum values $x$ can take. In this example we are solving the problem on the domain $0<x<1$.

In [9]:
geometry = {"rod": {x: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(1)}}}

We also need to provide the values of any parameters using the `pybamm.ParameterValues` class. This class accepts a dictionary of parameter names and values. Note that the name we provide is the string name of the parameters and not its symbol. For parameters that are functions we can define the function and pass it directly to `pybamm.ParameterValues`.

In [10]:
def applied_flux_function(A, omega):
    "Flux must return a function of time only"
    def applied_flux(t):
        return A * pybamm.sin(2 * np.pi * omega * t)
    
    return applied_flux


# Choose amplitude and frequency 
A = 2
omega = 5

# Define parameter values object
param = pybamm.ParameterValues(
    {
        "Applied flux": applied_flux_function(A, omega),
        "Applied flux": 1,
        "Diffusivity": 1,
        "Initial concentration": 1,
    },
)

Now that we have defined the geometry and provided the parameters values, we can process the model.

In [11]:
param.process_model(model)
param.process_geometry(geometry)

Before we disctretise the spatial operators, we must choose and mesh and a spatial method. Here we choose to use a uniformly spaced 1D mesh with 30 points, and discretise the equations in space using the finite volume method. The information about the mesh is stored in a `pybamm.Mesh` object, wheres the spatial methods are stored in a dictionary which maps domain names to a spatial method. This allows the user to discretise different (sub)domains in a problem using different spatial methods. All of this information goes into a `pybamm.Discretisation` object, which accepts a mesh and a dictionary of spatial methods.

In [12]:
submesh_types = {"rod": pybamm.Uniform1DSubMesh}
var_pts = {x: 8}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {"rod": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)

The model is then processed using the discretisation, turning the spaital operators into matrix-vector multiplications.

In [13]:
disc.process_model(model)

Now that the model has been discretised we are ready to solve. We must first choose a solver to use. For this model we choose the Scipy ODE solver, but other solvers are available in PyBaMM (see [here](https://pybamm.readthedocs.io/en/latest/source/solvers/index.html)). To solve the model, we use the method `solver.solve` which takes in a model and an array of times at which we would like the solution to be returned. Ths solution is then stored in the `solution` object. The times and states can be accessed with `solver.t` and `solver.y`.

In [15]:
# Choose solver
solver = pybamm.ScipySolver()

# Example: simulate for 10/omega seconds
simulation_time = 10/omega  # end time in seconds
npts = int(60 * simulation_time * omega)  # need enough timesteps to resolve output
t_eval = np.linspace(0, simulation_time, npts)
solution = solver.solve(model, t_eval)

We can create a slider plot of the results using `dynamic_plot`. We pass in the solution and a list of the variables we would like to plot

In [16]:
pybamm.dynamic_plot(solution, ["Concentration", "Surface concentration", "Applied flux"])

2022-09-13 15:21:21.171 - [WARNING] processed_variable.get_spatial_scale(501): No length scale set for rod. Using default of 1 [m].


interactive(children=(FloatSlider(value=0.0, description='t', max=2.0, step=0.02), Output()), _dom_classes=('w…

We can access the solution data by indexing into the solution object like a dictionary and calling entries attribute. Note this will raise a warning that we haven't set a length scale for our model. We can safely ignore this.

In [17]:
t = solution["Time"].entries  # array of size `Nt`
c = solution["Concentration"].entries  # array of size `Nx` by `Nt`

These arrays can be used to create a custom plot or as a part of other calculations.

## Extracting the matrix problem

We are interested in solving the problem when there is periodic forcing $j=j_0\mathrm{e}^{i\omega t}$. We look for a solution $c=y\mathrm{e}^{i\omega t}$ so that our problem becomes 

$$i \omega y = D y_{xx}, \quad \left. y_x \right|_{x=0} = 0, \quad \left. -Dy_x \right|_{x=1}=j_0,$$

In [18]:
y0 = solution.first_state
jac = model.jac_rhs_algebraic_eval(0, y0, []).full()

In [19]:
f = model.rhs_algebraic_eval(solution.t[0], solution.y[:, 0], [])

In [20]:
f

DM([0, 0, 0, 0, 0, 0, 0, -8])

In [21]:
jac

array([[ -64.,   64.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  64., -128.,   64.,    0.,    0.,    0.,    0.,    0.],
       [   0.,   64., -128.,   64.,    0.,    0.,    0.,    0.],
       [   0.,    0.,   64., -128.,   64.,    0.,    0.,    0.],
       [   0.,    0.,    0.,   64., -128.,   64.,    0.,    0.],
       [   0.,    0.,    0.,    0.,   64., -128.,   64.,    0.],
       [   0.,    0.,    0.,    0.,    0.,   64., -128.,   64.],
       [   0.,    0.,    0.,    0.,    0.,    0.,   64.,  -64.]])